In [8]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [9]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext


In [10]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import StringType, IntegerType,DoubleType, LongType, TimestampType
import pyspark.sql.functions as F
import numpy as np
import pandas as pd

In [11]:
# creating schema for the underlying data to enforce lazy evaluation
schema = StructType([
    StructField('User', IntegerType(), True),
    StructField('Card', IntegerType(), True),
    StructField('Year', IntegerType(), True),
    StructField('Month', IntegerType(), True),
    StructField('Day', IntegerType(), True),
    StructField('Time', StringType(), True),
    StructField('Amount', StringType(), True),
    StructField('Use Chip', StringType(), True),
    StructField('Merchant Name', LongType(), True),
    StructField('Merchant City', StringType(), True),
    StructField('Merchant State', StringType(), True),
    StructField('Zip', StringType(), True),
    StructField('MCC', IntegerType(), True),
    StructField('Errors?', StringType(), True),
    StructField('Is Fraud?', StringType(), True)
])

In [12]:
df = spark.read.csv('file:///home/talentum/shared/dataSource/credit_card_transactions-ibm_v2.csv', header = True, schema = schema)

In [13]:
#renaming column names to enter into parquet file as it does not support spaces, or special char in column names
new_columns = [
    "User",
    "Card",
    "Year",
    "Month",
    "Day",
    "Time",
    'Amount',
    'Use_Chip',
    'Merchant_Name',
    'Merchant_City',
    'Merchant_State',
    'Zip',
    'MCC',
    'Errors?',
    'Is_Fraud?'
]

# Apply the toDF() method to rename all columns
df_renamed = df.toDF(*new_columns)

In [ ]:
#saving data to hdfs and local dataStaging area in parquet data format

df_renamed.write.mode("overwrite").orc("file:///home/talentum/projectMaster/dataStaging/credit_card_transactions-ibm_v2.parquet")
df_renamed.write.mode("overwrite").orc('/user/talentum/projectMaster/dataStaging/credit_card_transactions-ibm_v2.parquet')